# Introduction to TensorFlow
   环境搭建：   
        conda create -n tensorflow python=3.5    
            source activate tensorflow    
            conda install pandas matplotlib jupyter notebook scipy scikit-learn

In [1]:
import tensorflow as tf
hello_constant = tf.constant('Hello World')
with tf.Session() as sess:
    output = sess.run(hello_constant)
    print(output)

b'Hello World'


In [4]:
import tensorflow as tf

def run():
	output = None
	x = tf.placeholder(tf.int32)

	with tf.Session() as sess:
		output = sess.run(x, feed_dict={x: 123})

	return output
out = run()
print(out)

array(123, dtype=int32)

In [9]:
import tensorflow as tf

x = tf.constant(10.)
y = tf.constant(2.)
z = tf.subtract(tf.divide(x, y), tf.constant(1.))

with tf.Session() as sess:
	output = sess.run(z)
print(output)

4.0


In [2]:
#Udacity Solution
import tensorflow as tf
x = tf.constant(10)
y = tf.constant(2)
z = tf.subtract(tf.divide(x, y), tf.cast(tf.constant(1), tf.float64))

with tf.Session() as sess:
    output = sess.run(z)
    print(output)

4.0


### TF class

In [6]:
import tensorflow as tf
x = tf.Variable(5)
init = tf.global_variables_initializer()

n_features  = 120
n_labels = 5
weights = tf.Variable(tf.truncated_normal((n_features, n_labels)))

n_labels = 5
bias = tf.Variable(tf.zeros(n_labels))

with tf.Session() as sess:
    print(sess.run(init))
    print(x)

None
<tf.Variable 'Variable_6:0' shape=() dtype=int32_ref>


### TensorFlow里的线性函数

In [10]:
def get_weights(n_features, n_labels):
    return tf.Variable(tf.truncated_normal((n_features, n_labels)))

def get_biases(n_labels):
    return tf.Variable(tf.zeros(n_labels))

def linear(input, w, b):
    return tf.add(tf.matmul(input, w), b)

In [12]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

def mnist_features_labels(n_labels):
    mnist_features = []
    mnist_labels = []
    
    mnist = input_data.read_data_sets("MNIST_data", one_hot=True)
    #读取一万张图片
    for mnist_feature, mnist_label in zip(*mnist.train.next_batch(10000)):
        #如果标签（0，1，2）存在，填加到列表
        #(0, 0, 0, 0, 1, 0, 0, 0, 0)[:n_labels].any() 返回False
        #(0, 1, 0, 0, 0, 0, 0, 0, 0)[:n_labels].any() 返回True
        if mnist_label[:n_labels].any():
            mnist_features.append(mnist_feature)
            mnist_labels.append(mnist_label[:n_labels])
            
    return mnist_features, mnist_labels

#fetures 28*28=784
n_features = 784

n_labels = 3

#Features and Labels
features = tf.placeholder(tf.float32)
labels = tf.placeholder(tf.float32)

#Weights and Biases
w = get_weights(n_features, n_labels)
b = get_biases(n_labels)

#Linear Function
logits = linear(features, w, b)

#Training data
train_features, train_labels = mnist_features_labels(n_labels)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    prediction = tf.nn.softmax(logits)
    
    cross_entropy = -tf.reduce_sum(labels * tf.log(prediction), reduction_indices=1)
    
    loss = tf.reduce_mean(cross_entropy)
    
    learning_rate = 0.08
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
    _, l = sess.run([optimizer, loss], feed_dict = {features: train_features, labels: train_labels})
    
print('Loss: {}'.format(l))
    
    

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Loss: 10.069314002990723


### TensorFlow Softmax

In [13]:
import tensorflow as tf

def run():
    output = None
    logit_data = [2.0, 1.0, 0.1]
    logits = tf.placeholder(tf.float32)
    
    softmax = tf.nn.softmax(logits)
    
    with tf.Session() as sess:
        output = sess.run(softmax, feed_dict = {logits: logit_data})
    return output
run()

array([ 0.65900117,  0.24243298,  0.09856589], dtype=float32)

### One-Hot Encoding
采用scikit-learn的LabelBinarizer实现

In [1]:
import numpy as np
from sklearn import preprocessing

labels = np.array([1,5,3,2,1,4,2,1,3])

#创建编码器
lb = preprocessing.LabelBinarizer()

#编码器找到类别并分配one-hot向量
lb.fit(labels)

#把目标(labels)转换成one-hot encoded向量
lb.transform(labels)

array([[1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0]])

### TensorFlow交叉熵

In [4]:
import tensorflow as tf

softmax_data = [0.7, 0.2, 0.1]
one_hot_data = [1.0, 0.0, 0.0]

softmax = tf.placeholder(tf.float32)
one_hot = tf.placeholder(tf.float32)

cross_entropy = -tf.reduce_sum(tf.multiply(one_hot, tf.log(softmax)))

with tf.Session() as sess:
    output = sess.run(cross_entropy, feed_dict = {softmax: softmax_data, one_hot: one_hot_data})
    print(output)

0.356675


### Mini-batch

In [17]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

n_input = 784 #输入维度
n_class = 10  #10个分类

#导入数据
mnist = input_data.read_data_sets("MNIST_data", one_hot = True)

train_features = mnist.train.images
test_features = mnist.test.images

train_labels = mnist.train.labels.astype(np.float32)
test_labels = mnist.test.labels.astype(np.float32)

weights = tf.Variable(tf.random_normal([n_input, n_class]))
biases = tf.Variable(tf.random_normal([n_class]))

print(train_features.shape, type(train_features), test_features.shape, test_labels.shape)

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    print(sess.run(weights))


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
(55000, 784) <class 'numpy.ndarray'> (10000, 784) (10000, 10)
[[-0.77327007  2.33807683 -1.66552627 ...,  0.49962536  0.51685196
  -1.38834178]
 [ 0.44412798 -1.23280287 -0.12093657 ..., -1.96390283 -0.58952963
   1.45713615]
 [-2.02368808 -1.36306787 -0.43683842 ...,  0.75053465  0.33054754
  -0.41696873]
 ..., 
 [-0.91441298 -0.30104902  0.39006051 ..., -0.1081707  -0.85398602
  -0.74767083]
 [ 1.76962876 -1.11978626 -0.2817426  ..., -0.11449924 -2.57422066
  -1.62547016]
 [-0.30794644  2.73987889  1.20813298 ...,  0.79665655 -1.05025947
  -1.5225538 ]]


### Implementing Minibatch with TF

In [9]:
import math
def batches(batch_size, features, labels):
    #return: Batches of (Features, Labels)
    a = []
    assert len(features) == len(labels)
    
    for idx in range(int(len(features)/batch_size) + 1):
        a.append([features[idx * batch_size: (idx + 1) * batch_size], labels[idx * batch_size: (idx + 1) * batch_size]])
    return a

from pprint import pprint
#pprint 提供了打印出任何python数据结构类和方法
#4 samples of features
example_features = [
    ['F11', 'F12', 'F13', 'F14'],
    ['F21', 'F22', 'F23', 'F24'],
    ['F31', 'F32', 'F33', 'F34'],
    ['F41', 'F42', 'F43', 'F44']]

#4 samples of labels
example_labels = [
    ['L11', 'L12'],
    ['L21', 'L22'],
    ['L31', 'L32'],
    ['L41', 'L42']]

#pprint prints data structures like 2d arrays, so they are easier to read
pprint(batches(3, example_features, example_labels))

[[[['F11', 'F12', 'F13', 'F14'],
   ['F21', 'F22', 'F23', 'F24'],
   ['F31', 'F32', 'F33', 'F34']],
  [['L11', 'L12'], ['L21', 'L22'], ['L31', 'L32']]],
 [[['F41', 'F42', 'F43', 'F44']], [['L41', 'L42']]]]


### MiniBatch on MNIST features

In [21]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import numpy as np

learning_rate = 0.001
n_input = 784
n_class = 10

mnist = input_data.read_data_sets("MNIST_data", one_hot = True)

train_features = mnist.train.images
test_features = mnist.test.images

train_labels = mnist.train.labels.astype(np.float32)
test_labels = mnist.test.labels.astype(np.float32)

features = tf.placeholder(tf.float32, [None, n_input])
labels = tf.placeholder(tf.float32, [None, n_class])

weights = tf.Variable(tf.random_normal([n_input, n_class]))
bias = tf.Variable(tf.random_normal([n_class]))

logits = tf.add(tf.matmul(features, weights), bias)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = labels))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(loss)

#计算准确率
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

#set batch size
batch_size = 128

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for batch_features, batch_labels in  batches(batch_size, train_features, train_labels):
        sess.run(optimizer, feed_dict = {features: batch_features, labels: batch_labels})
    
    test_accuracy = sess.run(accuracy, feed_dict = {features: test_features, labels: test_labels})
    
print("Test Accuracy: {}".format(test_accuracy))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Test Accuracy: 0.05849999934434891


### Epoch

In [25]:
from tensorflow .examples.tutorials.mnist import input_data
import tensorflow as tf
import numpy as np

def print_epoch_stats(epoch_i, sess, last_features, last_labels):
    #print cost and validation accuracy of an epoch
    
    current_cost = sess.run(
        cost,
        feed_dict={features: last_features, labels: last_labels})
    valid_accuracy = sess.run(
        accuracy,
        feed_dict={features: valid_features, labels: valid_labels})
    print('Epoch: {:<4} - Cost: {:<8.3} Valid Accuracy: {:<5.3}'.format(epoch_i, current_cost, valid_accuracy))
    #format()函数，格式限定符（语法是{}中带:号），填充与对齐，<,>分别是左对齐，右对齐，后面带宽度
    
n_input = 784
n_class = 10

mnist = input_data.read_data_sets("MNIST_data", one_hot = True)

train_features = mnist.train.images
valid_features = mnist.validation.images
test_features = mnist.test.images

train_labels = mnist.train.labels.astype(np.float32)
valid_labels = mnist.validation.labels.astype(np.float32)
test_labels = mnist.test.labels.astype(np.float32)

features = tf.placeholder(tf.float32, [None, 784])
labels = tf.placeholder(tf.float32, [None, 10])

weights = tf.Variable(tf.random_normal([n_input, n_class]))
bias = tf.Variable(tf.random_normal([n_class]))

logits = tf.add(tf.matmul(features, weights), bias)

learning_rate = tf.placeholder(tf.float32)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = labels))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(cost)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.global_variables_initializer()

batch_size = 128
epochs = 10
learn_rate = 0.001

train_batches = batches(batch_size, train_features, train_labels)

with tf.Session() as sess:
    sess.run(init)
    
    for epoch_i in range(epochs):
        for batch_features, batch_labels in train_batches:
            feed_dict = {
                features: batch_features,
                labels: batch_labels,
                learning_rate: learn_rate}
            sess.run(optimizer, feed_dict = feed_dict)
            
        print_epoch_stats(epoch_i, sess, batch_features, batch_labels)
        
    test_accuracy = sess.run(
        accuracy,
        feed_dict={features: test_features, labels: test_labels})
    
print('Test Accuracy: {}'.format(test_accuracy))
            

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Epoch: 0    - Cost: 13.1     Valid Accuracy: 0.132
Epoch: 1    - Cost: 11.8     Valid Accuracy: 0.151
Epoch: 2    - Cost: 10.8     Valid Accuracy: 0.162
Epoch: 3    - Cost: 10.0     Valid Accuracy: 0.169
Epoch: 4    - Cost: 9.32     Valid Accuracy: 0.178
Epoch: 5    - Cost: 8.74     Valid Accuracy: 0.186
Epoch: 6    - Cost: 8.24     Valid Accuracy: 0.197
Epoch: 7    - Cost: 7.82     Valid Accuracy: 0.214
Epoch: 8    - Cost: 7.45     Valid Accuracy: 0.229
Epoch: 9    - Cost: 7.12     Valid Accuracy: 0.244
Test Accuracy: 0.2402999997138977


In [26]:
from tensorflow .examples.tutorials.mnist import input_data
import tensorflow as tf
import numpy as np

def print_epoch_stats(epoch_i, sess, last_features, last_labels):
    #print cost and validation accuracy of an epoch
    
    current_cost = sess.run(
        cost,
        feed_dict={features: last_features, labels: last_labels})
    valid_accuracy = sess.run(
        accuracy,
        feed_dict={features: valid_features, labels: valid_labels})
    print('Epoch: {:<4} - Cost: {:<8.3} Valid Accuracy: {:<5.3}'.format(epoch_i, current_cost, valid_accuracy))
    #format()函数，格式限定符（语法是{}中带:号），填充与对齐，<,>分别是左对齐，右对齐，后面带宽度
    
n_input = 784
n_class = 10

mnist = input_data.read_data_sets("MNIST_data", one_hot = True)

train_features = mnist.train.images
valid_features = mnist.validation.images
test_features = mnist.test.images

train_labels = mnist.train.labels.astype(np.float32)
valid_labels = mnist.validation.labels.astype(np.float32)
test_labels = mnist.test.labels.astype(np.float32)

features = tf.placeholder(tf.float32, [None, 784])
labels = tf.placeholder(tf.float32, [None, 10])

weights = tf.Variable(tf.random_normal([n_input, n_class]))
bias = tf.Variable(tf.random_normal([n_class]))

logits = tf.add(tf.matmul(features, weights), bias)

learning_rate = tf.placeholder(tf.float32)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = labels))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(cost)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.global_variables_initializer()

batch_size = 128
epochs = 100
learn_rate = 0.001

train_batches = batches(batch_size, train_features, train_labels)

with tf.Session() as sess:
    sess.run(init)
    
    for epoch_i in range(epochs):
        for batch_features, batch_labels in train_batches:
            feed_dict = {
                features: batch_features,
                labels: batch_labels,
                learning_rate: learn_rate}
            sess.run(optimizer, feed_dict = feed_dict)
            
        print_epoch_stats(epoch_i, sess, batch_features, batch_labels)
        
    test_accuracy = sess.run(
        accuracy,
        feed_dict={features: test_features, labels: test_labels})
    
print('Test Accuracy: {}'.format(test_accuracy))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Epoch: 0    - Cost: 12.1     Valid Accuracy: 0.142
Epoch: 1    - Cost: 10.9     Valid Accuracy: 0.161
Epoch: 2    - Cost: 10.0     Valid Accuracy: 0.178
Epoch: 3    - Cost: 9.37     Valid Accuracy: 0.198
Epoch: 4    - Cost: 8.82     Valid Accuracy: 0.215
Epoch: 5    - Cost: 8.33     Valid Accuracy: 0.233
Epoch: 6    - Cost: 7.91     Valid Accuracy: 0.251
Epoch: 7    - Cost: 7.54     Valid Accuracy: 0.266
Epoch: 8    - Cost: 7.21     Valid Accuracy: 0.285
Epoch: 9    - Cost: 6.92     Valid Accuracy: 0.298
Epoch: 10   - Cost: 6.66     Valid Accuracy: 0.317
Epoch: 11   - Cost: 6.42     Valid Accuracy: 0.327
Epoch: 12   - Cost: 6.21     Valid Accuracy: 0.339
Epoch: 13   - Cost: 6.0      Valid Accuracy: 0.352
Epoch: 14   - Cost: 5.82     Valid Accuracy: 0.362
Epoch: 15   - Cost: 5.64     Valid Accura